In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import statsmodels.api as sm

# Load the dataset
surveyV1_df = pd.read_csv("D:/ExportFile/TVSEP2022SurveyV1.csv", low_memory=False)
shocks_df = pd.read_csv("D:/ExportFile/shocks.csv",low_memory=False)
shocks_details_df = pd.read_csv("D:/ExportFile/shocks_detail.csv",low_memory=False)

surveyV1Col = list(surveyV1_df.columns)
shocksCol = list(shocks_df.columns)
shocksDeCol = list(shocks_details_df.columns)



#data = data[shockRelated]

In [6]:
print(shocksCol)

['Unnamed: 0', 'interview__key', 'interview__id', 'shocks__id', 'shocks_detail__id', 'v31102d', 'v31102a__0', 'v31102a__1', 'v31102a__2', 'v31102a__3', 'v31102a__4', 'v31102a__5', 'v31102a__6', 'v31102a__7', 'v31102a__8', 'v31102a__9', 'v31102a__10', 'v31102a__11', 'v31102a__12', 'v31102a__13', 'v31102a__14', 'v31102a__15', 'v31102a__16', 'v31102a__17', 'v31102a__18', 'v31102a__19', 'rowcode_shock', 'v31103', 'v31103a', 'v31103c', 'v31104', 'v31105a', 'v31105b', 'v31106a', 'shockmoneyloss', 'v31107', 'v31108a__1', 'v31108a__2', 'v31108a__3', 'v31108a__4', 'v31108a__5', 'v31108a__6', 'v31108a__7', 'v31108__40', 'v31108__41', 'v31108__42', 'v31108__43', 'v31108__3', 'v31108__4', 'v31108__5', 'v31108__6', 'v31108__7', 'v31108__8', 'v31108__9', 'v31108__10', 'v31108__11', 'v31108__12', 'v31108__13', 'v31108__14', 'v31108__15', 'v31108__16', 'v31108__17', 'v31108__18', 'v31108__19', 'v31108__20', 'v31108__21', 'v31108__22', 'v31108__23', 'v31108__24', 'v31108__25', 'v31108__26', 'v31108__27

In [ ]:
# List of DataFrames to merge
dfs = [surveyV1_df, shocks_df, shocks_details_df]

# Merge all DataFrames on 'interview_key'
# Merge all DataFrames on 'interview_key'
merged_df = reduce(lambda left, right: pd.merge(left, right, on="interview_key", how="outer"), dfs)
#merged_df = reduce(lambda left, right: pd.merge(left, right, on="interview__key", how="outer"), dfs)
merged_df.fillna(0, inplace=True)

In [ ]:
mList = list(merged_df)
print(mList)

In [ ]:
# Select numerical columns for descriptive statistics
"""
shockRelated = [
'v31102__6', 'v31102__10', 'v31102__11', 'v31102__63', 'v31102__16', 'v31102__55', 'v31102__77', 'v31102__5', 'v31102__8', 'v31102__1',
 'v31102__2', 'v31102__3', 'v31102__24', 'v31102__46', 'v31102__62', 'v31102__18', 'v31102__21', 'v31102__22', 'v31102__70', 'v31102__90',
 'v31102b','v31105a', 'v31105b','v31106a','v31108a__1', 'v31108a__2', 'v31108a__3', 'v31108a__4', 'v31108a__5', 'v31108a__6', 'v31108a__7',
 'v31108__40', 'v31108__41', 'v31108__42', 'v31108__43', 'v31108__3', 'v31108__4', 'v31108__5', 'v31108__6', 'v31108__7', 'v31108__8', 'v31108__9', 
 'v31108__10', 'v31108__11', 'v31108__12', 'v31108__13', 'v31108__14', 'v31108__15', 'v31108__16', 'v31108__17', 'v31108__18', 'v31108__19', 
 'v31108__20', 'v31108__21', 'v31108__22', 'v31108__23', 'v31108__24', 'v31108__25', 'v31108__26', 'v31108__27', 'v31108__28', 'v31108__29', 
 'v31108__30', 'v31108__31', 'v31108__48', 'v31108__49', 'v31108__50', 'v31108__51', 'v31108__52', 'v31108__53', 'v31108__54', 'v31108__55', 
 'v31108__62', 'v31108__63', 'v31108__90', 'v31108__98']
 """

shockIndex_df = merged_df.iloc[:,"v31102__6"]
shockIndex_df.head()

In [ ]:
shockIndex_df = data.iloc[:,[shockRelated]]

In [ ]:

#shockRelated.head()

data.head()

# Compute descriptive statistics
#descriptive_stats = data[shockRelated].describe()
#print(descriptive_stats)

In [ ]:
# Select variables for correlation analysis
correlation_columns = ["PYIncCap", "hh_size", "head_age", "total_asset", "owned_land"]

# Compute correlation matrix
correlation_matrix = data[correlation_columns].corr()
print(correlation_matrix)

In [ ]:


# Heatmap of correlations
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

# Scatter plot: Income vs. Household Size
sns.scatterplot(x="hh_size", y="PYIncCap", data=data)
plt.title("Income vs. Household Size")
plt.xlabel("Household Size")
plt.ylabel("Per Capita Income")
plt.show()

In [ ]:
#Testing the corelation between no_shock and total_asset
# Select relevant columns
shocks_corelelation = ["no_shocks", "total_asset"]

# Drop rows with missing values in the selected columns
data_cleaned = data[shocks_corelelation].dropna()

# Descriptive statistics
print(data_cleaned.describe())

# Correlation analysis
correlation = data_cleaned["no_shocks"].corr(data_cleaned["total_asset"])
print(f"Correlation between no_shocks and total_asset: {correlation}")

# Scatter plot
plt.scatter(data_cleaned["no_shocks"], data_cleaned["total_asset"], alpha=0.5)
plt.title("Relationship between No. of Shocks and Total Assets")
plt.xlabel("Number of Shocks")
plt.ylabel("Total Assets")
plt.show()


In [ ]:
# Define the independent (X) and dependent (Y) variables
X = data_cleaned["no_shocks"]
Y = data_cleaned["total_asset"]

# Add a constant to the independent variable
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the regression summary
print(model.summary())